In [ ]:
! pip install transformers
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.9 MB/s eta 0:00:

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel,BertForMaskedLM
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
import torch
from transformers import BertForMaskedLM, BertTokenizer
from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/CSCI544NLPFinalProject/finetuning/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# labeled_comments = pd.read_csv(path + '/jigsaw_pos_dataset.csv')
# input_sentences = labeled_comments['comment_text'].tolist()
labeled_comments  = pd.read_csv(path + '/processed_training_data_3.csv')
input_sentences = labeled_comments[labeled_comments['label'] == 1.0]
input_sentences = input_sentences['preprocessed_text']
input_sentences = [sentence for sentence in input_sentences if type(sentence)==str and  len(sentence.split()) > 2]

In [ ]:
input_sentences[0]

'cocksucker before you piss around on my work'

In [ ]:
with open(path + '/bad_word.txt', 'r') as f:
    not_predict = f.read().split('\n')

In [ ]:
with open(path + '/jigsaw_predictions.txt', 'r') as f:
    labels = f.read()

In [ ]:
labels = [float(x) for x in labels.split('\n')]

In [ ]:
# with open(path + '/masked_sentences.txt', 'r') as f:
#   input_sentences = f.readlines()

In [ ]:
# input_sentences = [x[:-1].replace('<', '[').replace('>', ']') for x in input_sentences]

In [ ]:
import pandas as pd
import os

df = pd.read_csv(path + "/train.csv")

bad_words=[]
with open(path + "/bad_word.txt") as f:
    for line in f.readlines():
        bad_words.append(line.split('\n')[0])

bad_words_set = set(bad_words)

In [ ]:
masked_sentences = []

for sentence in input_sentences:
    flag = False
    words = sentence.split()
    for j, word in enumerate(words):
        if word.lower() in bad_words_set:
            flag = True
            words[j] = "[MASK]"
    if flag:
        masked_sentence = " ".join(words)
        masked_sentences.append(masked_sentence) 
    else:
        pass

In [ ]:
masked_sentences[0:10]

['[MASK] before you [MASK] around on my work',
 'bye don t look come or think of comming back [MASK]',
 'you are [MASK] or antisemmitian archangel white tiger meow greetingshhh uh there are two way why you do erased my comment about ww2 that holocaust wa brutally slaying of [MASK] and not [MASK] gypsy [MASK] anyone 1 if you are anti semitian than shave your head bald and go to the skinhead meeting 2 if you doubt word of the [MASK] that homosexuality is a deadly sin make a pentagram tatoo on your forehead go to the satanistic mass with your [MASK] pal 3 first and last warning you [MASK] [MASK] i won t appreciate if any more [MASK] shwain would write in my page i don t wish to talk to you anymore beware of the dark side',
 '[MASK] your filthy mother in the as dry',
 'get [MASK] up get fuckeeed up got a drink that you cant put down get [MASK] up get [MASK] up i m [MASK] up right now',
 '[MASK] peace of [MASK] stop deleting my stuff [MASK] go [MASK] and fall in a [MASK] go to [MASK]',
 'my

In [ ]:
path+'/fine_tuned_model'

'/content/drive/MyDrive/CSCI544NLPProject/finetuning/data/fine_tuned_model'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(path+'/fine_tuned_model')
model = BertForMaskedLM.from_pretrained(path+'/fine_tuned_model')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
masked_sentences = ['<mask> <mask> <mask>',
 '<mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> show <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> <mask> <mask> <mask> <mask> <mask> good <mask> <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> <mask> <mask> <mask> <mask> <mask> good show <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> <mask> <mask> fuck <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> <mask> <mask> fuck <mask> <mask> <mask> show <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> <mask> <mask> fuck <mask> <mask> good <mask> <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> <mask> <mask> fuck <mask> <mask> good show <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> <mask> <mask> show',
 '<mask> <mask> <mask> show <mask>',
 '<mask> <mask> a <mask> <mask>',
 '<mask> <mask> a <mask> show',
 '<mask> <mask> a show <mask>',
 '<mask> <mask> good <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> good <mask> <mask> <mask> <mask> <mask> show <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> good <mask> fuck <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> good <mask> fuck <mask> <mask> <mask> show <mask> <mask> <mask> <mask> <mask>',
 '<mask> <mask> show <mask> <mask>',
 '<mask> <mask> that <mask> <mask>',
 '<mask> <mask> that <mask> show',
 '<mask> <mask> that show <mask>',
 '<mask> a <mask> <mask> <mask>',
 '<mask> a <mask> <mask> show',
 '<mask> a <mask> show <mask>',
 '<mask> a show <mask> <mask>',
 '<mask> that <mask> <mask> <mask>',
 '<mask> that <mask> <mask> show',
 '<mask> that <mask> show <mask>',
 '<mask> that show <mask> <mask>',
 'fuck <mask> <mask>']

In [ ]:
masked_sentences = [x.replace('<mask>','[MASK]') for x in masked_sentences]
masked_sentences[10:20]

['[MASK] [MASK] [MASK] [MASK] show',
 '[MASK] [MASK] [MASK] show [MASK]',
 '[MASK] [MASK] a [MASK] [MASK]',
 '[MASK] [MASK] a [MASK] show',
 '[MASK] [MASK] a show [MASK]',
 '[MASK] [MASK] good [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]',
 '[MASK] [MASK] good [MASK] [MASK] [MASK] [MASK] [MASK] show [MASK] [MASK] [MASK] [MASK] [MASK]',
 '[MASK] [MASK] good [MASK] fuck [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]',
 '[MASK] [MASK] good [MASK] fuck [MASK] [MASK] [MASK] show [MASK] [MASK] [MASK] [MASK] [MASK]',
 '[MASK] [MASK] show [MASK] [MASK]']

In [ ]:
model.eval()

for input_text in masked_sentences[0:10]:
  # input_text = preprocess_text(input_text)
  print(input_text)
  tokens = tokenizer(input_text, return_tensors="pt")
  input_ids = tokens["input_ids"]
  attention_mask = tokens["attention_mask"]

  with torch.no_grad():
      outputs = model(input_ids, attention_mask=attention_mask)
      predictions = outputs.logits

  masked_indices = torch.where(input_ids == tokenizer.mask_token_id)[1]

  predicted_tokens = []
  for masked_index in masked_indices:
      predicted_index = torch.argmax(predictions[0, masked_index]).item()
      # print([tokenizer.convert_ids_to_tokens(x) for x in torch.topk(predictions[0, masked_index], k=5)[1]])
      top_values, top_indices = torch.topk(predictions[0, masked_index], k=5)
      top_predicted_tokens = tokenizer.convert_ids_to_tokens(top_indices.tolist())

      resulting_word = ""
      for i in top_predicted_tokens: 
        if i in not_predict: 
            continue
        else: 
            resulting_word = i
            break
      predicted_tokens.append(resulting_word)
      
      # print(top_predicted_tokens)
      # predicted_token = tokenizer.convert_ids_to_tokens(predicted_index)
      # predicted_tokens.append(predicted_token)

  print("Predicted tokens:", predicted_tokens)
  index = 0 
  resulting_sentence = ""
  words = input_text.split()
  for i in range(len(words)):
      if words[i]=='[MASK]': 
          words[i] = predicted_tokens[index]
          index+=1 
  resulting_sentence = ' '.join(words)
  print(resulting_sentence)

[MASK] before you [MASK] around on my work
Predicted tokens: ['"', 'mess']
" before you mess around on my work
bye don t look come or think of comming back [MASK]
Predicted tokens: ['.']
bye don t look come or think of comming back .
you are [MASK] or antisemmitian archangel white tiger meow greetingshhh uh there are two way why you do erased my comment about ww2 that holocaust wa brutally slaying of [MASK] and not [MASK] gypsy [MASK] anyone 1 if you are anti semitian than shave your head bald and go to the skinhead meeting 2 if you doubt word of the [MASK] that homosexuality is a deadly sin make a pentagram tatoo on your forehead go to the satanistic mass with your [MASK] pal 3 first and last warning you [MASK] [MASK] i won t appreciate if any more [MASK] shwain would write in my page i don t wish to talk to you anymore beware of the dark side
Predicted tokens: ['fascist', 'jews', 'a', 'or', 'gospel', 'old', 'are', 'no', 'of']
you are fascist or antisemmitian archangel white tiger meo

In [ ]:
undesired_tokens = set(not_predict)

undesired_token_ids = {tokenizer.encode(token, add_special_tokens=False)[0] for token in undesired_tokens}

In [ ]:
def tokenize_and_prepare(sentence):
    tokens = tokenizer(sentence, return_tensors="pt", padding='max_length', truncation=True, max_length=128)
    input_ids = tokens["input_ids"].squeeze()
    attention_mask = tokens["attention_mask"].squeeze()
    
    # Replace -100 in the labels tensor with the original token ID for non-masked tokens
    labels = input_ids.clone().detach()
    labels[labels != tokenizer.mask_token_id] = -100

    return {
        "input_ids": torch.tensor(input_ids),
        "attention_mask": torch.tensor(attention_mask),
        "labels": torch.tensor(labels)
    }

prepared_data = [tokenize_and_prepare(sentence) for sentence in masked_sentences]

<ipython-input-26-26dd7da2b118>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(input_ids),
<ipython-input-26-26dd7da2b118>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(attention_mask),
<ipython-input-26-26dd7da2b118>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(labels)


In [ ]:
import pickle

with open(path + "/prepared_data.pkl", "wb") as file:
    pickle.dump(prepared_data, file)

In [ ]:
with open(path + "/prepared_data.pkl", "rb") as file:
    loaded_prepared_data = pickle.load(file)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch.nn.functional as F
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from datasets import DatasetDict


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def custom_loss(outputs, labels, undesirable_token_ids, penalty_weight=10.0):
    logits = outputs.logits
    loss = F.cross_entropy(logits.view(-1, logits.shape[-1]), labels.view(-1), ignore_index=-100)

    # Compute the penalty mask using torch.isin()
    predicted_tokens = logits.argmax(-1)
    penalty_mask = torch.isin(predicted_tokens, torch.tensor(list(undesirable_token_ids)).to(device))

    penalty = (penalty_mask * penalty_weight).sum()
    return loss + penalty


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = custom_loss(outputs, labels, undesired_token_ids)

        return (loss, outputs) if return_outputs else loss

dataset = Dataset.from_dict({k: [d[k] for d in prepared_data] for k in prepared_data[0]})

train_val_split = dataset.train_test_split(test_size=0.2)
train_val_split = DatasetDict(train_val_split)

training_args = TrainingArguments(
    output_dir='./results',          # Output directory for the model and other artifacts
    num_train_epochs=3,              # Total number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    save_total_limit=3,              # Maximum number of checkpoints to keep
    evaluation_strategy="steps",     # Evaluation strategy (steps or epoch)
    eval_steps=100,                  # Evaluate the model every 100 steps
    logging_steps=10,                # Log the training progress every 10 steps
)

custom_trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_val_split['train'],
    eval_dataset=train_val_split['test'],
    tokenizer=tokenizer
)

custom_trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
100,0.036400,0.016333
200,0.002600,0.001305
300,0.000900,0.000420
400,0.000400,0.000170
500,0.000200,0.000074
600,0.000100,0.000034
700,0.000100,0.000017
800,0.000100,0.000009
900,0.000000,0.000004
1000,0.000000,0.000003


TrainOutput(global_step=3969, training_loss=0.1675969316562506, metrics={'train_runtime': 1873.4259, 'train_samples_per_second': 16.947, 'train_steps_per_second': 2.119, 'total_flos': 2089122400396800.0, 'train_loss': 0.1675969316562506, 'epoch': 3.0})

In [ ]:
model.save_pretrained(path + "/fine_tuned_model")
tokenizer.save_pretrained(path + "/fine_tuned_model")

('/content/drive/MyDrive/CSCI544_NLP_Project-transfering_to_asus_lappy/finetuning/data/fine_tuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/CSCI544_NLP_Project-transfering_to_asus_lappy/finetuning/data/fine_tuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/CSCI544_NLP_Project-transfering_to_asus_lappy/finetuning/data/fine_tuned_model/vocab.txt',
 '/content/drive/MyDrive/CSCI544_NLP_Project-transfering_to_asus_lappy/finetuning/data/fine_tuned_model/added_tokens.json')

In [ ]:
from datasets import DatasetDict


train_val_split = dataset.train_test_split(test_size=0.2)
train_val_split = DatasetDict(train_val_split)

In [ ]:
train_val_split

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10583
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2646
    })
})

In [ ]:
from transformers import BertForMaskedLM, BertTokenizer, BertConfig, Trainer, TrainingArguments
from torch.optim import AdamW
from torch.utils.data import DataLoader
import torch
from datasets import Dataset
from datasets import DatasetDict

# dataset = Dataset.from_dict({k: torch.tensor([d[k] for d in prepared_data]) for k in prepared_data[0]})
dataset = Dataset.from_dict({k: [d[k] for d in prepared_data] for k in prepared_data[0]})
print(dataset)
train_val_split = dataset.train_test_split(test_size=0.2)
train_val_split = DatasetDict(train_val_split)

dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
undesired_token_weight = 100  # You can set a higher value to penalize the undesired tokens more

for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        optimizer.zero_grad()

        print(batch['input_ids'])
        print(batch)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Modify the loss for undesired tokens
        logits = outputs.logits
        for undesired_token_id in undesired_token_ids:
            undesired_mask = (labels == undesired_token_id).float()
            undesired_loss = torch.sum(logits * undesired_mask, dim=-1)
            loss += undesired_token_weight * undesired_loss

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}: Loss = {loss.item()}")

TypeError: ignored

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define a list of words that the model should not predict

# Load a pre-trained BERT model and tokenizer
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model = model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the optimizer and loss function for fine-tuning
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Define a list of input sentences and labels
# sentences = input_sentences[:100]
sentences = input_sentences

labels = []
for sentence in sentences:
    label_sentence = []
    try:
        if len(sentence)>1:
            sentence = sentence.split()
        for word in sentence:
            if word in not_predict:
                label_sentence.append(0)
            else:
                label_sentence.append(1)
        labels.append(label_sentence)
    except:
        print(sentence)


# Convert the input sentences to input features
input_features = []
for sentence in sentences:
    # Tokenize the input sentence
    tokenized_sentence = tokenizer.encode(sentence, add_special_tokens=True)
    # Create a list of labels for each token in the input sentence
    sentence_labels = []
    for token in tokenized_sentence:
        word = tokenizer.decode([token]).strip()
        if word in not_predict:
            sentence_labels.append(0)  # label 0 for blacklisted words
        else:
            sentence_labels.append(1)  # label 1 for other words
    # Append the input features and labels to the input_features list
    input_features.append((torch.tensor(tokenized_sentence).to(device), torch.tensor(sentence_labels).to(device)))

# Fine-tune the BERT model on the input features and labels
for epoch in range(10):
    running_loss = 0.0
    for input_ids, labels in tqdm(input_features):
        optimizer.zero_grad()
        outputs = model(input_ids.unsqueeze(0), labels=labels.unsqueeze(0))
        loss, prediction_scores = outputs[:2]
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("Epoch {} loss: {:.4f}".format(epoch+1, running_loss/len(input_features)))




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 100/100 [00:15<00:00,  6.52it/s]


Epoch 1 loss: 8.4241


100%|██████████| 100/100 [00:15<00:00,  6.42it/s]


Epoch 2 loss: 0.4234


100%|██████████| 100/100 [00:15<00:00,  6.51it/s]


...

Epoch 8 loss: 0.0664


100%|██████████| 100/100 [00:16<00:00,  6.20it/s]


Epoch 9 loss: 0.0635


100%|██████████| 100/100 [00:16<00:00,  6.25it/s]

Epoch 10 loss: 0.0614


In [ ]:
# Evaluate the fine-tuned model on some example input sentences
test_sentences = [
    "here comes the beginning of another fucking recession",
    "he enjoys a fucking orange juice.",
    "she bought some nipples at the store.",
    "we went to the park to play sucker."
]
for sentence in test_sentences:
    # Tokenize the input sentence
    tokenized_sentence = tokenizer.encode(sentence, add_special_tokens=True)
    # Create a list of labels for each token in the input sentence
    sentence_labels = []
    # Convert the input sentence to a PyTorch tensor and predict the masked words
    input_ids = torch.tensor(tokenized_sentence).to(device)
    print(input_ids)
    with torch.no_grad():
        outputs = model(input_ids.unsqueeze(0))
        print(outputs)
        prediction_scores = outputs[0]
    # Decode the predicted masked words and print the result
    predicted_tokens = torch.argmax(prediction_scores, dim=-1).squeeze().tolist()
    print(predicted_tokens)
    predicted_words = tokenizer.decode(predicted_tokens).split()
    print("Input sentence:", sentence)
    print("Predicted words:", predicted_words)


tensor([  101,  2182,  3310,  1996,  2927,  1997,  2178,  8239, 19396,   102],
       device='cuda:0')
MaskedLMOutput(loss=None, logits=tensor([[[15.3306, 19.9276,  8.0185,  ...,  7.0004,  4.2338,  3.4028],
         [15.1143, 19.7127,  7.7288,  ...,  6.8165,  4.2824,  3.0219],
         [15.0954, 19.6774,  7.7642,  ...,  6.8457,  4.3665,  3.0776],
         ...,
         [15.2913, 19.8027,  7.8640,  ...,  6.9566,  4.3831,  3.1447],
         [15.4877, 20.0838,  8.1257,  ...,  7.0895,  4.6768,  3.3599],
         [14.0324, 18.7651,  6.8386,  ...,  6.1382,  3.7032,  2.3645]]],
       device='cuda:0'), hidden_states=None, attentions=None)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Input sentence: here comes the beginning of another fucking recession
Predicted words: ['[unused0]', '[unused0]', '[unused0]', '[unused0]', '[unused0]', '[unused0]', '[unused0]', '[unused0]', '[unused0]', '[unused0]']
tensor([  101,  2002, 15646,  1037,  8239,  4589, 10869,  1012,   102],
       device='cuda:0')
MaskedLMOutput(